# **绘制实时音频频谱（Real-time Audio Signal Frequency Spectrum）**

**使用 Python 获取音频信号并做 FFT 分析，使用 CodeLab Adapter Linda 发送数据，使用 Scratch 实时画图**

In [ ]:
import pyaudio
import numpy as np
import time

# 通过 CodeLab Adapter Linda 将 python 处理后的音频数据发送给 Scratch 画图
from codelab_adapter_client import AdapterNode
class MyNode(AdapterNode):
    NODE_ID = "linda/test"

    def __init__(self):
        super().__init__()
        
node = MyNode()
node.receive_loop_as_thread()


buff_size = 8820                    
wanted_num_of_bins = 12              
fs = 44100
pa = pyaudio.PyAudio()
stream = pa.open(format=pyaudio.paInt16, channels=1, rate=fs, input=True, frames_per_buffer=int(buff_size))


end_index = []
for i in np.arange(12):    
    end_index.append(int((buff_size/2)/2**i)-1)
end_index.reverse()

    
start_index = [0]
for i in end_index[:-1]:
    start_index.append(i+1)        

#n=0

while True:

    block = stream.read(int(buff_size))
    data = np.fromstring(block, dtype=np.int16)

    x = np.array(data)/(2**15)
    seg_len = len(x)

    
    X = np.abs(np.fft.fft(x))[0:int(seg_len/2)]


    X2 = []
    for i in np.arange(12):
        XX = np.mean(X[start_index[i]:end_index[i]])        
        X2.append(XX)
        i=i+1
    
    outlist = ['X2_12',X2]
    
    node.linda_out(outlist)

    # n = n + 1





<br>

## **参考：**

[Audio Handling Basics: Process Audio Files In Command-Line or Python](https://hackernoon.com/audio-handling-basics-how-to-process-audio-files-using-python-cli-jo283u3y)

[Frequency spectrum of sound using Pyaudio, NumPy, and Matplotlib](https://blog.yjl.im/2012/11/frequency-spectrum-of-sound-using.html)

[Realtime FFT Audio Visualization with Python](https://swharden.com/blog/2013-05-09-realtime-fft-audio-visualization-with-python/)

[Recording Stereo Audio on a Raspberry Pi](https://makersportal.com/blog/recording-stereo-audio-on-a-raspberry-pi)